In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,Input,Layer

In [ ]:
from tensorflow.keras.layers import Concatenate,BatchNormalization

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#Model Building

In [ ]:
class Inception_cell(Layer):
    def __init__(self,num_filters,activation='relu',padding='same',kernel_initializer='he_normal',**kwargs):
        super().__init__(**kwargs)
        self.num_filters=num_filters
        self.activation=activation
        self.padding=padding
        self.kernel_initializer=kernel_initializer
        self.conv1=Conv2D(self.num_filters,(3,3),activation=self.activation,padding=self.padding,kernel_initializer=self.kernel_initializer)
        self.conv2=Conv2D(self.num_filters,(5,5),activation=self.activation,padding=self.padding,kernel_initializer=self.kernel_initializer)
        self.conv3=Conv2D(self.num_filters,(7,7),activation=self.activation,padding=self.padding,kernel_initializer=self.kernel_initializer)
        
    def call(self,input):
        x1=self.conv1(input)
        x2=self.conv2(input)
        x3=self.conv3(input)
        out=Concatenate(axis=-1)([x1,x2,x3])
        return out
        
    def get_config(self):
        config=super().get_config()
        config.update({
            'num_filters':self.num_filters,
            'activation':self.activation,
            'padding':self.padding,
            'kernel_initializer':self.kernel_initializer
        })
        return config


        

In [ ]:
class SkipConnection(Layer):
    def __init__(self,num_filters,kernel_size,activation='relu',padding='same',kernel_initializer='he_normal',**kwargs):
        super().__init__(**kwargs)
        self.num_filters=num_filters
        self.activation=activation
        self.padding=padding
        self.kernel_size=kernel_size
        self.kernel_initializer=kernel_initializer
        self.conv=Conv2D(self.num_filters,(7,7),activation=self.activation,padding=self.padding,kernel_initializer=self.kernel_initializer)
        self.batch_normalization=BatchNormalization()
        
    def call(self,input):
        x=self.conv(input)
        x=self.batch_normalization(x)
        x=Concatenate(axis=-1)([x,input])
        return x
        
    def get_config(self):
        config=super().get_config()
        config.update({
            'num_filters':self.num_filters,
            'activation':self.activation,
            'padding':self.padding,
            'kernel_size':self.kernel_size,
            'kernel_initializer':self.kernel_initializer
        })
        return config
        
        

In [ ]:
def build_model(input_shape):
    inp=Input(shape=input_shape,dtype=tf.float32)
    # Inception cell
    x=Inception_cell(num_filters=8)(inp)
    x=MaxPooling2D((2,2),padding='same')(x)
    x=Inception_cell(num_filters=16)(x)
    x=MaxPooling2D((2,2),padding='same')(x)
    
    # Skip Connection Cell
    x=SkipConnection(num_filters=16,kernel_size=(3,3))(x)
    x=MaxPooling2D((2,2),padding='same')(x)
    x=SkipConnection(num_filters=32,kernel_size=(3,3))(x)
    x=MaxPooling2D((2,2),padding='same')(x)

    # Final Layers
    x=Flatten()(x)
    x=Dense(units=512,activation='relu',kernel_initializer='he_normal')(x)
    x=Dense(units=128,activation='relu',kernel_initializer='he_normal')(x)
    x=Dropout(0.2)(x)
    x=Dense(units=32,activation='relu',kernel_initializer='he_normal')(x)
    x=Dense(units=8,activation='relu',kernel_initializer='he_normal')(x)
    x=Dense(units=1,activation='sigmoid',kernel_initializer='glorot_normal')(x)
    
    # Model
    model=Model(inputs=inp,outputs=x)
    
    return model
    
    
    
    
    

In [ ]:
model=build_model(input_shape=(256,256,3))

In [ ]:
model.summary()

In [ ]:
import math
from tensorflow.keras.callbacks import LearningRateScheduler

class CosineAnnealingScheduler(LearningRateScheduler):
    
    def __init__(self,initial_learning_rate,total_epochs):
        self.total_epochs=total_epochs
        self.initial_lr=initial_learning_rate
        super().__init__(self.scheduler_function,verbose=0)
        
    def scheduler_function(self,epoch,lr):
        return 0.5*self.initial_lr*(1+math.cos(math.pi*(epoch/self.total_epochs)))
        
    def get_config(self):
        config=super().get_config()
        config.update(
            {
                'initial_lr':self.initial_lr,
                'total_epochs':self.total_epochs
            }
        )
        return config
       
        

#Model Training

In [ ]:
optimizer=Adam(learning_rate=0.01,beta_1=0.96,beta_2=0.98)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs=2
batch_size=100
callback1=CosineAnnealingScheduler(initial_learning_rate=0.01,total_epochs=epochs)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255.0)

In [ ]:
train_dir=r"D:\Codeutsav\Images"

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=batch_size,
    class_mode='sparse'
)

In [ ]:
train_generator.class_indices

In [ ]:
history=model.fit(train_generator,callbacks=[callback1],batch_size=batch_size,epochs=epochs)

In [ ]:
model.save('drug_finder_img.h5')